# Setup

In [1]:
!pip install -r requirements.txt

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


In [2]:
# General libraries 
from datetime import date
from datetime import *
import pandas as pd
import numpy as np
import warnings
import logging
import mlflow
import sys
import os

# Plots
from matplotlib.offsetbox import TextArea, AnnotationBbox
import matplotlib.pyplot as plt
import networkx as nx

# Stats
from scipy.stats import norm

# LP
import pulp

ModuleNotFoundError: No module named 'mlflow'

In [ ]:
# Load assets
from simulator_utils import *

# Directory

In [4]:
# Set cd to parent directory
os.chdir("/Users/ignasipascual/Documents/GitHub/Optimizator")
cwd = os.getcwd()
print("Current Working Directory:", cwd)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/ignasipascual/Documents/GitHub/LightGBM-Forecaster'

# Assets

In [ ]:
# Load assets
from optimizator import *
from plotter import *

# Parameters

In [ ]:
# Set random seed
np.random.seed(42)

# Sales parameters
supply_params = (200, 50)
sell_in_params = (150, 40)

# Initial inventory
initial_inventory = 1000

# Start simulator

In [ ]:
# Initialize simulator
sim = Simulator()

# Show
sim

## Simulate data

In [ ]:
# Example usage:
plants = ['plant_1']
warehouses = ['warehouse_1', 'warehouse_2']
markets = ['market_1', 'market_2']
products = ['product_1']
weeks = [202501 + i for i in range(10)]

# Example warehouse-market mapping
warehouse_market_map = {
    "warehouse_1": ["market_1"], 
    "warehouse_2": ["market_1", "market_2"], 
}

In [ ]:
# Generate the dataset
sc_data = sim.generate_supply_chain_data(plants, warehouses, markets, products, weeks, warehouse_market_map=warehouse_market_map)

# Show
sc_data.head()

plant,warehouse,market,product,week
plant_1,warehouse_1,market_1,product_1,202501
plant_1,warehouse_1,market_1,product_1,202502
plant_1,warehouse_1,market_1,product_1,202503
plant_1,warehouse_1,market_1,product_1,202504
plant_1,warehouse_1,market_1,product_1,202505
plant_1,warehouse_1,market_1,product_1,202506
plant_1,warehouse_1,market_1,product_1,202507
plant_1,warehouse_1,market_1,product_1,202508
plant_1,warehouse_1,market_1,product_1,202509
plant_1,warehouse_1,market_1,product_1,202510


In [ ]:
# Printing distinct values
print("Distinct plants:", sc_data['plant'].unique())
print("Distinct warehouses:", sc_data['warehouse'].unique())
print("Distinct markets:", sc_data['market'].unique())
print("Distinct products:", sc_data['product'].unique())

Distinct plants: ['plant_1']
Distinct warehouses: ['warehouse_1' 'warehouse_2']
Distinct markets: ['market_1' 'market_2']
Distinct products: ['product_1']


## Generate flows

In [ ]:
# Simulate data
sc_data = sim.simulate_data_normal(
    supply_params,
    sell_in_params
)

# Show
sc_data.head()

plant,warehouse,market,product,week,supply,sell_in
plant_1,warehouse_1,market_1,product_1,202501,225.0,126.0
plant_1,warehouse_1,market_1,product_1,202502,193.0,224.0
plant_1,warehouse_1,market_1,product_1,202503,232.0,149.0
plant_1,warehouse_1,market_1,product_1,202504,276.0,108.0
plant_1,warehouse_1,market_1,product_1,202505,188.0,183.0
plant_1,warehouse_1,market_1,product_1,202506,188.0,101.0
plant_1,warehouse_1,market_1,product_1,202507,279.0,158.0
plant_1,warehouse_1,market_1,product_1,202508,238.0,72.0
plant_1,warehouse_1,market_1,product_1,202509,177.0,97.0
plant_1,warehouse_1,market_1,product_1,202510,227.0,158.0


## Inventory

In [ ]:
# Calculate inventory
sc_data = sim.calculate_rolling_inventory(initial_inventory)

# Show
sc_data.head()

plant,warehouse,market,product,week,supply,sell_in,inventory
plant_1,warehouse_1,market_1,product_1,202501,225.0,126.0,1099.0
plant_1,warehouse_1,market_1,product_1,202502,193.0,224.0,1068.0
plant_1,warehouse_1,market_1,product_1,202503,232.0,149.0,1151.0
plant_1,warehouse_1,market_1,product_1,202504,276.0,108.0,1319.0
plant_1,warehouse_1,market_1,product_1,202505,188.0,183.0,1324.0
plant_1,warehouse_1,market_1,product_1,202506,188.0,101.0,1411.0
plant_1,warehouse_1,market_1,product_1,202507,279.0,158.0,1532.0
plant_1,warehouse_1,market_1,product_1,202508,238.0,72.0,1698.0
plant_1,warehouse_1,market_1,product_1,202509,177.0,97.0,1778.0
plant_1,warehouse_1,market_1,product_1,202510,227.0,158.0,1847.0


## Lead times

In [ ]:
# Add lead times
sc_data_lead_times = sim.set_lead_times()

# Show
sc_data_lead_times.head()

plant,warehouse,market,product,week,supply,sell_in,inventory,lead_time
plant_1,warehouse_1,market_1,product_1,202501,225.0,126.0,1099.0,2
plant_1,warehouse_1,market_1,product_1,202502,193.0,224.0,1068.0,2
plant_1,warehouse_1,market_1,product_1,202503,232.0,149.0,1151.0,2
plant_1,warehouse_1,market_1,product_1,202504,276.0,108.0,1319.0,2
plant_1,warehouse_1,market_1,product_1,202505,188.0,183.0,1324.0,2
plant_1,warehouse_1,market_1,product_1,202506,188.0,101.0,1411.0,2
plant_1,warehouse_1,market_1,product_1,202507,279.0,158.0,1532.0,2
plant_1,warehouse_1,market_1,product_1,202508,238.0,72.0,1698.0,2
plant_1,warehouse_1,market_1,product_1,202509,177.0,97.0,1778.0,2
plant_1,warehouse_1,market_1,product_1,202510,227.0,158.0,1847.0,2


## Summary

In [ ]:
# Get summary
summary = sim.get_summary()

# Show
summary

inventory                          ... safety_stock                   
                  min     max    mean     std  ...          min    max   mean  std
warehouse                                      ...                                
warehouse_1    1068.0  1847.0  1422.7  282.66  ...        75.16  75.16  75.16  0.0
warehouse_2     997.0  1520.0  1266.4  182.01  ...        85.11  85.11  85.11  0.0

[2 rows x 20 columns]

# Save data

In [ ]:
# Write csv
sc_data_lead_times.to_csv('data/simulated_data.csv', index=False)